In [1]:
import time as time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tag.util import untag

In [2]:
# ressources necessaires
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beb4e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\beb4e\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\beb4e\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beb4e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### fonctions utilitaires génériques, parametrages affichage

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv("Questions.csv")
df = df.drop(columns="Id", axis=1)
df

,Title,Body,Tags
0,Java 14 records and arrays,<p>Given the following code:</p>\n\n<pre><code...,<java><arrays><java-14><java-record>
1,Blue flickering stripes on my MBP with Chrome,"<p>On my new Macbook Pro 16 late 2019, with Ca...",<google-chrome><graphics><macos-catalina>
2,How can I create a DependsOn relation between ...,<p>I am currently using the <em>aws-cdk</em> (...,<amazon-web-services><amazon-ec2><amazon-rds><...
3,on colab - class_weight is causing a ValueErro...,<p>i'm running a CNN with keras sequential on ...,<python><tensorflow><keras><google-colaborator...
4,How to pass a custom function inside a ForEach...,<p>I can't find a way to pass the function. Ju...,<powershell><powershell-core>
...,...,...,...
8064,Daemon is stopping immediately JVM garbage col...,<p>When I build multi module java project usin...,<java><gradle>
8065,Difference between Mono/Flux.fromCallable and ...,<p>I usually have the requirement of generatin...,<spring-webflux><project-reactor>
8066,How to compare generic structs in C++?,<p>I want to compare structs in a generic way ...,<c++><templates><struct><padding><memcmp>
8067,How to set up a custom HTTP error in Kubernetes,<p>I want to create a custom 403 error page.\n...,<kubernetes><kubernetes-ingress>


In [5]:
df['Body'][0]

'<p>Given the following code:</p>\n\n<pre><code>public static void main(String[] args) {\n    record Foo(int[] ints){}\n\n    var ints = new int[]{1, 2};\n    var foo = new Foo(ints);\n    System.out.println(foo); // Foo[ints=[I@6433a2]\n    System.out.println(new Foo(new int[]{1,2}).equals(new Foo(new int[]{1,2}))); // false\n    System.out.println(new Foo(ints).equals(new Foo(ints))); //true\n    System.out.println(foo.equals(foo)); // true\n}\n</code></pre>\n\n<p>It seems, obviously, that array\'s <code>toString</code>, <code>equals</code> methods are used (instead of static methods, <code>Arrays::equals</code>,<code>Arrays::deepEquals</code>  or <code>Array::toString</code>).</p>\n\n<p>So I guess Java 14 Records (<a href="https://openjdk.java.net/jeps/359" rel="noreferrer">JEP 359</a>) don\'t work too well with arrays, the respective methods have to be generated with an IDE (which at least in IntelliJ, by default generates "useful" methods, i.e. they use the static methods in <code

In [6]:
%%time

df['Body'] = df['Body'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df['Title'] = df['Title'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

Wall time: 4.32 s


In [7]:
df['Body'][0]

'Given the following code:\npublic static void main(String[] args) {\n    record Foo(int[] ints){}\n\n    var ints = new int[]{1, 2};\n    var foo = new Foo(ints);\n    System.out.println(foo); // Foo[ints=[I@6433a2]\n    System.out.println(new Foo(new int[]{1,2}).equals(new Foo(new int[]{1,2}))); // false\n    System.out.println(new Foo(ints).equals(new Foo(ints))); //true\n    System.out.println(foo.equals(foo)); // true\n}\n\nIt seems, obviously, that array\'s toString, equals methods are used (instead of static methods, Arrays::equals,Arrays::deepEquals  or Array::toString).\nSo I guess Java 14 Records (JEP 359) don\'t work too well with arrays, the respective methods have to be generated with an IDE (which at least in IntelliJ, by default generates "useful" methods, i.e. they use the static methods in Arrays). \nOr is there any other solution?\n'

In [8]:
# stemming puis lemmization

In [9]:
def clean_text(text):
    text = re.sub(r"\'", " ", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r"\n", " ", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r"\xa0", " ", text) # match all literal non-breakable space pattern then replace them by a single whitespace
    text = re.sub('\s+', ' ', text) # match all one or more whitespace then replace them by a single whitespace
    text = text.strip(' ')
    return text

In [10]:
%%time

df['Title'] = df['Title'].apply(lambda x: clean_text(x)) 
df['Body'] = df['Body'].apply(lambda x: clean_text(x))

Wall time: 509 ms


In [11]:
df['Body'][0]

'Given the following code: public static void main(String[] args) { record Foo(int[] ints){} var ints = new int[]{1, 2}; var foo = new Foo(ints); System.out.println(foo); // Foo[ints=[I@6433a2] System.out.println(new Foo(new int[]{1,2}).equals(new Foo(new int[]{1,2}))); // false System.out.println(new Foo(ints).equals(new Foo(ints))); //true System.out.println(foo.equals(foo)); // true } It seems, obviously, that array s toString, equals methods are used (instead of static methods, Arrays::equals,Arrays::deepEquals or Array::toString). So I guess Java 14 Records (JEP 359) don t work too well with arrays, the respective methods have to be generated with an IDE (which at least in IntelliJ, by default generates "useful" methods, i.e. they use the static methods in Arrays). Or is there any other solution?'

In [12]:
%%time

df['Title'] = df['Title'].str.lower()
df['Body'] = df['Body'].str.lower()

Wall time: 29 ms


In [13]:
df['Body'][0]

'given the following code: public static void main(string[] args) { record foo(int[] ints){} var ints = new int[]{1, 2}; var foo = new foo(ints); system.out.println(foo); // foo[ints=[i@6433a2] system.out.println(new foo(new int[]{1,2}).equals(new foo(new int[]{1,2}))); // false system.out.println(new foo(ints).equals(new foo(ints))); //true system.out.println(foo.equals(foo)); // true } it seems, obviously, that array s tostring, equals methods are used (instead of static methods, arrays::equals,arrays::deepequals or array::tostring). so i guess java 14 records (jep 359) don t work too well with arrays, the respective methods have to be generated with an ide (which at least in intellij, by default generates "useful" methods, i.e. they use the static methods in arrays). or is there any other solution?'

In [14]:
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'

def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac)) 


# on fait pas celui là, trop violent (supprime tout les nombres, meme ceux collés ou dans un mot)
def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [15]:
%%time

df['Title'] = df['Title'].apply(lambda x: remove_non_alphabetical_character(x))
df['Body'] = df['Body'].apply(lambda x: remove_non_alphabetical_character(x)) 

Wall time: 862 ms


In [16]:
df['Body'][0]

'given the following code public static void main string args record foo int ints var ints new int var foo new foo ints system out println foo foo ints i a system out println new foo new int equals new foo new int false system out println new foo ints equals new foo ints true system out println foo equals foo true it seems obviously that array s tostring equals methods are used instead of static methods arrays equals arrays deepequals or array tostring so i guess java records jep don t work too well with arrays the respective methods have to be generated with an ide which at least in intellij by default generates useful methods i e they use the static methods in arrays or is there any other solution '

In [17]:
token = ToktokTokenizer()
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    """remove common words in english by using nltk.corpus's list"""
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [18]:
%%time

df['Title'] = df['Title'].apply(lambda x: remove_stopwords(x))
df['Body'] = df['Body'].apply(lambda x: remove_stopwords(x)) 

Wall time: 1.74 s


In [19]:
df['Body'][0]

'given following code public static void main string args record foo int ints var ints new int var foo new foo ints system println foo foo ints system println new foo new int equals new foo new int false system println new foo ints equals new foo ints true system println foo equals foo true seems obviously array tostring equals methods used instead static methods arrays equals arrays deepequals array tostring guess java records jep work well arrays respective methods generated ide least intellij default generates useful methods e use static methods arrays solution'

In [20]:
stemmer = PorterStemmer()

def stem_text(text):
    """Stem the text"""
    words = nltk.word_tokenize(text) # tokenize the text then return a list of tuple (token, nltk_tag)
    stem_text = []
    for word in words:
        stem_text.append(stemmer.stem(word)) # Stem each words
    return " ".join(stem_text) # Return the text untokenize

In [21]:
%%time

# df['Title'] = df['Title'].apply(lambda x: stem_text(x)) 
# df['Body'] = df['Body'].apply(lambda x: stem_text(x)) 

Wall time: 0 ns


In [22]:
df['Body'][0]

'given following code public static void main string args record foo int ints var ints new int var foo new foo ints system println foo foo ints system println new foo new int equals new foo new int false system println new foo ints equals new foo ints true system println foo equals foo true seems obviously array tostring equals methods used instead static methods arrays equals arrays deepequals array tostring guess java records jep work well arrays respective methods generated ide least intellij default generates useful methods e use static methods arrays solution'

In [23]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    """Lemmatize the text by using tag """
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))  # tokenize the text then return a list of tuple (token, nltk_tag)
    lemmatized_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'a')) # Lemmatisze adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'v')) # Lemmatisze verbs
        elif tag.startswith('N'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'n')) # Lemmatisze nouns
        elif tag.startswith('R'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'r')) # Lemmatisze adverbs
        else:
            lemmatized_text.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatization
    return " ".join(lemmatized_text) # Return the text untokenize

In [24]:
%%time

df['Title'] = df['Title'].apply(lambda x: lemmatize_text(x))
df['Body'] = df['Body'].apply(lambda x: lemmatize_text(x))

Wall time: 1min 17s


In [25]:
df['Body'][0]

'give follow code public static void main string args record foo int ints var ints new int var foo new foo ints system println foo foo ints system println new foo new int equal new foo new int false system println new foo ints equal new foo ints true system println foo equal foo true seem obviously array tostring equal method use instead static method array equal array deepequals array tostring guess java record jep work well arrays respective method generate ide least intellij default generates useful method e use static method array solution'

In [26]:
df['Text'] = df['Title'] + ' ' + df['Body'] 

In [27]:
df.to_csv('questions_clean.csv', encoding='utf-8', errors='surrogatepass')